<a href="https://colab.research.google.com/github/akhich551995/my_first_repository-akhil/blob/master/GPS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
gps= pd.read_csv('/content/googleplaystore.csv')


FileNotFoundError: ignored

In [ ]:
#gps.describe()
#gps.columns
#gps.head(5)
#gps.tail(5)
gps.info()
#gps.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10841 entries, 0 to 10840
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   App             10841 non-null  object 
 1   Category        10841 non-null  object 
 2   Rating          9367 non-null   float64
 3   Reviews         10841 non-null  object 
 4   Size            10841 non-null  object 
 5   Installs        10841 non-null  object 
 6   Type            10840 non-null  object 
 7   Price           10841 non-null  object 
 8   Content Rating  10840 non-null  object 
 9   Genres          10841 non-null  object 
 10  Last Updated    10841 non-null  object 
 11  Current Ver     10833 non-null  object 
 12  Android Ver     10838 non-null  object 
dtypes: float64(1), object(12)
memory usage: 1.1+ MB


Google play store data has 13 columns with dtypes:float64(1), object(12)
dimension of the data is 10841*13



1.   Apps are divided into different categories
2.   we have multiple columns containing numeric data, they have non numeric data type. this causes problems with data processing.




# Data Preprocessing


Lets find out no of unique apps and unique categories in the data set.

In [ ]:
unique_apps= len(gps["App"].unique())
unique_cat= len(gps["Category"].unique())
print(unique_apps,unique_cat)

9660 34


so we have 9660 unique apps which are of 34 different categories.




In [ ]:
gps.drop_duplicates(inplace=True)
gps.shape

(10358, 13)

In [ ]:
gps.isnull().sum()

App                  0
Category             0
Rating            1465
Reviews              0
Size                 0
Installs             0
Type                 1
Price                0
Content Rating       1
Genres               0
Last Updated         0
Current Ver          8
Android Ver          3
dtype: int64

We found 1465 apps doesnt have rating or have NA value. Only rating has major misisng data , 

In [3]:
gps["rating"]

NameError: ignored